In [22]:
import pandas as pd
from pathlib import Path
import shutil

In [23]:
# old sentence info sheet - currently using sentence_info.csv inside sentence_transcription folder/repo
'''sentence_df = pd.read_csv("LASR_sentence_recording_material - forOishani.csv")
sentence_df = sentence_df.rename(columns={'Unnamed: 2': 'probability'})
sentence_df[['sent_num', 'prob_type']] = sentence_df['Sentence ID'].str.split('_', expand=True)
sentence_df = sentence_df[['Sentence ID', 'sent_num', 'prob_type', 'Expected Transcription', 'probability']]
sentence_df.head()'''

'sentence_df = pd.read_csv("LASR_sentence_recording_material - forOishani.csv")\nsentence_df = sentence_df.rename(columns={\'Unnamed: 2\': \'probability\'})\nsentence_df[[\'sent_num\', \'prob_type\']] = sentence_df[\'Sentence ID\'].str.split(\'_\', expand=True)\nsentence_df = sentence_df[[\'Sentence ID\', \'sent_num\', \'prob_type\', \'Expected Transcription\', \'probability\']]\nsentence_df.head()'

In [24]:
sentence_df = pd.read_csv("sentence_info.csv")
sentence_df = sentence_df.rename(columns={'sentence_type': 'probability'})
sentence_df[['sent_num', 'prob_type']] = sentence_df['sentence_id'].str.split('_', expand=True)
sentence_df = sentence_df[['sentence_id', 'sent_num', 'prob_type', 'probability', 'reference', 
                           'expected_transcription', 'acceptable_transcription']]
sentence_df.head()

,sentence_id,sent_num,prob_type,probability,reference,expected_transcription,acceptable_transcription
0,1_1,1,1,high prob,The monkey ate the banana.,the monkey ate the banana,NaN
1,1_2,1,2,low prob,The monkey ate the shark.,the monkey ate the shark,NaN
2,1_3,1,3,nonword,The runkey ate the shark.,the runkey ate the shark,NaN
3,2_1,2,1,high prob,Mom likes to use the elevator instead of the s...,mom likes to use the elevator instead of the s...,NaN
4,2_2,2,2,nonword,Pom likes to use the elevator instead of the d...,pom likes to use the elevator instead of the d...,NaN


In [25]:
# quick check for corrections
sentence_df[sentence_df['sentence_id'] == "96_3"]

,sentence_id,sent_num,prob_type,probability,reference,expected_transcription,acceptable_transcription
287,96_3,96,3,high prob,"In the woods lived a big, growly bear.",in the woods lived a big growly bear,['in the woods lived a big growly beer']


### Getting Low Probability wav Files into Participant Folders

In [27]:
# get low prob sentences from df
low_ids = set(
    sentence_df.loc[sentence_df["probability"].str.strip().str.lower().eq("low prob"), "sentence_id"]
      .astype(str)
      .str.strip()
)

print(f"Found {len(low_ids)} low-prob Sentence IDs")


Found 100 low-prob Sentence IDs


In [29]:
# find participant directories in root folder
root = Path("/Users/cogsci-lasrlab1/Desktop/MTAA_recording")  

# commented out to work with new participants only - this is the original code
participant_dirs = sorted([p for p in root.iterdir() if p.is_dir() and p.name.startswith("t")])
print(participant_dirs)

# commented out to re-run all folders
# to add new participant prob folders ONLY
'''
participant_names = ['t78', 't79',
                   't80', 't82',
                   't83', 't8']

participant_dirs = [root / name for name in participant_names]
'''

[PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t1'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t10'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t12'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t13'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t15'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t17'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t18'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t19'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t2'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t20'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t21'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t22'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t23'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t24'), PosixPath('/Users/cogsci-lasrlab1/Desktop/MTAA_recording/t25'), PosixPath('/Users/cogsci-lasrlab1/Desktop

"\nparticipant_names = ['t78', 't79',\n                   't80', 't82',\n                   't83', 't8']\n\nparticipant_dirs = [root / name for name in participant_names]\n"

In [42]:
# re-running
# copy matching relevant wav files from the df
for pdir in participant_dirs:
    out_dir = pdir / f"{pdir.name}_low_prob"
    out_dir.mkdir(exist_ok=True)

    copied = 0
    for wav_path in pdir.glob("*.wav"):  # only wav files directly inside t1/ etc.
        fname = wav_path.name

        # Match any low-prob Sentence ID in the filename.
        # Using "sauce_{sid}_" to avoid accidental partial matches.
        if any(f"sauce_{sid}_" in fname for sid in low_ids):
            shutil.copy2(wav_path, out_dir / wav_path.name)
            copied += 1

### Creating probability split folders in each participant folder

In [33]:
def get_prob_ids(prob): # gets you the set of sentence ids for each probability input
    prob_ids = set(
    sentence_df.loc[sentence_df["probability"].str.strip().str.lower().eq(prob), "sentence_id"]
      .astype(str)
      .str.strip()
    )
    return prob_ids

In [34]:
high_prob_ids = get_prob_ids("high prob")
low_prob_ids = get_prob_ids("low prob")
nonword_ids = get_prob_ids("nonword")

In [35]:
print("low prob".split()[0])

low


In [40]:
# copy name matched files regardless of extension
# to respective folder for participant
def make_prob_folder(participant_dirs, prob):
    prob_ids = get_prob_ids(prob)
    
    for pdir in participant_dirs:
        # for low and high prob:
        # out_dir = pdir / f"{pdir.name}_{prob.split()[0]}_{prob.split()[1]}_all"

        # for nonword
        out_dir = pdir / f"{pdir.name}_{prob}_all"
        
        out_dir.mkdir(exist_ok=True)
    
        copied = 0
        for path in pdir.iterdir():  # files directly inside t1/ etc.
            fname = path.name
    
            # Match prob and Sentence ID in the filename.
            # Using "sauce_{sid}_" to avoid accidental partial matches.
            if any(f"sauce_{sid}_" in fname for sid in prob_ids):
                shutil.copy2(path, out_dir / path.name)
                copied += 1

In [37]:
make_prob_folder(participant_dirs, "high prob")

In [38]:
make_prob_folder(participant_dirs, "low prob")

In [41]:
make_prob_folder(participant_dirs, "nonword")